# Forecasting with Prophet in Python on BQ Data

In [ ]:
# pip install prophet

In [ ]:
# pip install google-auth

In [ ]:
#pip install google-cloud-storage

In [ ]:
#pip install bigquery

In [ ]:
#pip install google-cloud-bigquery-storage

## 1. Import Libraries

In [ ]:
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation

import os
import datetime
import pandas as pd
import numpy as np
import datetime as dt

import google.auth
from google.cloud import storage
from google.cloud import bigquery
from google.cloud import bigquery_storage

from scipy import stats as st

import matplotlib.pyplot as plt

import random
from datetime import datetime, timedelta

import logging
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

## 2. Access dataset

### 2.1 Example how to access BQ data from a Jupyter Notebook

In [ ]:
# load the bigquery credentials to google cloud

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "YOUR JSON CREDENTIALS"

# bigquery

credentials, your_project_id = google.auth.default(
scopes=["https://www.googleapis.com/auth/cloud-platform"])

# create clients

bqclient = bigquery.Client(credentials=credentials, project=your_project_id,)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [ ]:
# connect to the bigquery table and create dataframe

string = """
select *
FROM dummy_table
"""

df = (
    bqclient.query(string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
df.head()

### 2.2 Example accessing CSV

In [ ]:
df_fb=pd.read_csv("CSV_name.csv")

### 2.3 Example creating fake data

In [ ]:
# will be using this one as an example

In [ ]:
# function to generate fake data

def generate_fake_data():
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2023, 12, 31)
    current_date = start_date

    data = {'Date': [], 'Downloads': [], 'Country': []}
    downloads_base = 1000  

    while current_date <= end_date:
        
        # add seasonality and trend components
        
        seasonality = 100 * (1 + 0.5 * (1 + (current_date.month % 12) / 12))
        trend = 10 * (current_date.year - 2022)

        # generate random noise
        
        noise = random.uniform(-50, 50)

        # calculate total downloads
        
        downloads = int(downloads_base + seasonality + trend + noise)

        # append data to the dictionary
        
        data['Date'].append(current_date)
        data['Downloads'].append(downloads)
        data['Country'].append('US')

        # move to the next day
        
        current_date += timedelta(days=1)

    return data

In [ ]:
# generate the data

fake_data = generate_fake_data()

In [ ]:
# create df

df_fb = pd.DataFrame(fake_data)

In [ ]:
# confirm

df_fb.head()

## 3. Using Prophet for Forecasting

In [ ]:
# The code can be used for other applications besides 4.
# No real data is used, so it is all hypothetical

### 3.1 Defining placeholders

In [ ]:
# defining placeholders and limited to two weeks in the context of the application in 4. and the data itself

country=["US"]
start_date="2023-12-05"
update_date="2023-12-18"
end_date="2023-12-31"

### 3.2 Define all data dataset and train dataset

In [ ]:
# Define the function that returns all data dataset and train dataset
# Function line 1: select country
# Function line 2: select the two variables for Prophet, time and dimension to forecast
# Function line 3: create columns designed for Prophet, "ds" for date, "y" for the dimension to forecast (following documentation)
# Function line 4: make sure only positive numbers are available
# Funtion line 5: convert "ds" to date time as needed to apply Prophet
# Function line 6: define the train set

In [ ]:
def train_global_df (country):
    df=df_fb[(df_fb["Country"]==country)]
    df=df[["Date","Downloads"]]
    df.columns=["ds","y"]
    df['y'] = pd.to_numeric(df['y'], errors='coerce')
    df=df[df["y"]>0]
    df['ds']=pd.to_datetime(df['ds'])
    df_train=df[(df["ds"]<update_date)]
    return df, df_train

In [ ]:
# call train set as example

train_global_df("US")[1]

### 3.3 Define the Prophet method, create the forecast and an accuracy evaluation (MAPE: Mean Absolute Percentage Error)

In [ ]:
# Define the function that returns the forecast and the MAPE
# Function line 1: define Prophet call and its hyperparameter tuning
# Function line 2: add country holidays
# Function line 3: fit the train set
# Function line 4: set the forecast length to 30 days (option to decrease runing time)
# Funtion line 5: create future dataframe based on previous information
# Function line 6: apply the forecast
# Function line 7 to 10: manipulate to compare forecast with actual numbers
# Function line 11: define MAPE call

In [ ]:
def forecast (country):
    fb=Prophet(changepoint_prior_scale=0.15, seasonality_prior_scale=0.1)
    fb.add_country_holidays(country_name=country)
    fb.fit(train_global_df(country)[1])
    forecast_length=95
    future=fb.make_future_dataframe (periods=forecast_length)
    forecast=fb.predict(future)
    forecast=forecast[['ds','yhat']]
    plot=forecast.merge(train_global_df(country)[0],how="left")
    plot=plot[plot["ds"]<end_date]
    plot["diff_forecast-actuals"]=plot["y"]-plot["yhat"]
    mape=performance_metrics (cross_validation(fb, horizon = '30 days')).iloc[10,4]
    return plot, mape

In [ ]:
# check forecast

forecast("US")[0]

### 3.4 Define a static forecast and MAPE accuracy measure to avoid running the function multiple times

In [ ]:
# forecast

plot_total=forecast("US")[0]
plot_total

In [ ]:
# mape

mape=forecast("US")[1]
mape

### 3.5 Visualize the Forecast vs Actuals

In [ ]:
# Define the function that creates the graph
# Line 1: Define the size
# Line 2: Define the scatter plot for the forecasted cases
# Line 3: Define the scatter plot for the true/ actual cases
# Line 4: Create a vertical line with the update date/ until the date we defined the train dataset
# Following lines: Define legend, lable and title

In [ ]:
def data_viz_total ():
    plt.figure(figsize=(10,7))
    plt.scatter(plot_total['ds'], plot_total['yhat'], label='Forecasted Cases')
    plt.scatter(plot_total['ds'], plot_total['y'], label='True Cases')
   # plt.axvline(x=update_date,color="red")
    plt.legend(loc="upper left")
    plt.xlabel("Date", size=10)
    plt.ylabel("Downloads", size=10)
    plt.title("Forecast vs Actual Downloads US", size=15)
    return plt

In [ ]:
# call the function

data_viz_total ()

## 4. Application Example: Daily Uplift from ASO Update

In [ ]:
# The goal is to compare the forecast of downloads after an ASO optimization to measure impact
# Impact would be measured by comparing the actuals and the forecast
# The forecast in this context would represent the downloads expected without any update
# The difference between actuals and forecast would represent the uplift (or not) from the update - was it successful or not?

In [ ]:
# fix plot function 

plot=plot_total[(plot_total["ds"]>=start_date) & (plot_total["ds"]<=end_date)]

In [ ]:
# funcion fb prophet

def impact_measurement(country):
    plot2weeks=plot[(plot["ds"]>=update_date) & (plot["ds"]<=end_date)]
    forecast_table=pd.DataFrame(index=[0],columns=["Country","Forecast","Actuals","Daily Uplift","MAPE"])
    forecast_table["Country"]=country
    forecast_table["Forecast"]=sum(plot2weeks["yhat"])/len(plot2weeks["yhat"])
    forecast_table["Actuals"]=sum(plot2weeks["y"])/len((plot2weeks["y"]))
    forecast_table["Daily Uplift"]=sum(plot2weeks['diff_forecast-actuals'])/len(plot2weeks['diff_forecast-actuals'])
    forecast_table["MAPE"]=mape
    forecast_table=forecast_table.round(2)
    return forecast_table

In [ ]:
impact_measurement ("US")

## 5. Eliminate Outliers

In [ ]:
# define a dataframe based on the train set

df_outliers=train_global_df("US")[1]

In [ ]:
# outliers function for search

def outliers ():
    df_outliers_stats=df_outliers.describe().transpose()
    iqr_out=df_outliers_stats.loc["y"]["75%"]-df_outliers_stats.loc["y"]["25%"]
    df_out=df_outliers[(df_outliers["y"]>=(df_outliers_stats.loc["y"]["25%"]-(1.5*iqr_out))) & (df_outliers["y"]<=(df_outliers_stats.loc["y"]["75%"]+(1.5*iqr_out)))]
    return df_out

In [ ]:
# call function

outliers ()

## 6. Hyperparameter tuning

In [ ]:
import itertools

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1,0.15, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0,5.0, 10.0],
}

# Generate all combinations of parameters

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters

for params in all_params:
    fb = Prophet(**params).fit(train_global_df ("US")[1])  # Fit model with given params
    df_cv = cross_validation(fb, horizon='30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters

tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

In [ ]:
# check the best ones

best_params = all_params[np.argmin(rmses)]
print(best_params)